In [34]:
# Create Workspace
# Create Compute Target
# Resgistering Data
# Creating conda Environment

# Create Experiment
# Register model in workspace
# Deploying the model
# Prepare your test data
# Real time scoring using RestAPI Call

In [35]:
from azureml.core import Workspace, ComputeTarget, Datastore, Dataset, Experiment
from azureml.core.compute import AmlCompute

In [36]:
# Details of your AML workspace
subscription_id = "0f644ff4-434a-4446-a414-16ddb2b713ee"
resource_group = "rg-sbihackathon"
workspace = "ml-demo-explore"

In [37]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [40]:
# Creating Workspace
from azureml.core.workspace import WorkspaceException
# from azureml.core.authentication import InteractiveLoginAuthentication

try:
    # ws = Workspace.from_config() #fetches details from 'config.json'
    # ia = InteractiveLoginAuthentication(tenant_id='7c0c36f5-af83-4c24-8844-9962e0163719')
    # Alternate way to configure workspace
    ws = Workspace.get(subscription_id='0f644ff4-434a-4446-a414-16ddb2b713ee',
                    resource_group='rg-sbihackathon',
                    name='ml-demo-explore')
    
    print('Workspace already exist!')
    # print('Workspace already exist!',ws, sep='\n')
    print('\nCompute Targets:')
    if ws.compute_targets != {} or len(ws.compute_targets) != 0:
        for compute_name in ws.compute_targets:
            compute = ws.compute_targets[compute_name]
            # compute = ComputeTarget._get(ws, compute_name)
            print("\t",compute.name, ":", compute.type)
    else:
        print("\tNone")
    print('Datastores:')
    if ws.datastores != {} or len(ws.datastores) != 0:
        for datastore_name in ws.datastores:
            # datastore = ws.datastores[datastore_name]
            datastore = Datastore.get(ws, datastore_name)
            print("\t",datastore.name, ':', datastore.datastore_type)
    else:
        print("\tNone")   
    print('Datasets:')
    if len(list(ws.datasets.keys())) != 0:
        for dataset_name in list(ws.datasets.keys()):
            dataset = Dataset.get_by_name(ws, dataset_name)
            print('\t',dataset.name)
    else:
        print("\tNone")
    print('Experiments:')
    # print(ws.experiments)
    if ws.experiments != {} or len(ws.experiments) != 0:
        for experiment_name in ws.experiments:
            # experiment = Experiment._get_base_info_dict() (ws, experiment_name)
            experiment = ws.experiments[experiment_name]
            print("\t",experiment.name, ':', experiment.workspace.name)
    else:
        print("\tNone")
        
    print("\n",ws)
except WorkspaceException as workspaceException:
    print("[ERROR]: ", workspaceException)
    # Creates the workspace
    # ws=Workspace.create(workspace_name, 
    #                 resource_group=resource_name,
    #                 create_resource_group=True,
    #                 subscription_id=subscriptionID,
    #                 location="East US")
    # ws.write_config('.azureml')
except Exception as e:
    print("[ERROR]: ", e)

Workspace already exist!

Compute Targets:
	 buf-batch-cluster : AmlCompute
Datastores:
	 azureml_globaldatasets : AzureBlob
	 azureml : AzureBlob
	 workspaceartifactstore : AzureBlob
	 workspaceworkingdirectory : AzureFile
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob
Datasets:
	 banktermdeposit
	 buf-v1
Experiments:
	 exp-day1-no-1 : ml-demo-explore
	 exp-day2-no1 : ml-demo-explore
	 exp-day2-no2 : ml-demo-explore

 Workspace.create(name='ml-demo-explore', subscription_id='0f644ff4-434a-4446-a414-16ddb2b713ee', resource_group='rg-sbihackathon')


In [ ]:
# Compute Target Details
aml_compute_target = "buf-batch-cluster" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'demo_expirement'

In [ ]:
# Create Compute Target
from azureml.core.compute_target import ComputeTargetException

try:
    # aml_compute = AmlCompute(ws, aml_compute_target)
    aml_compute = ml_client.compute.get(aml_compute_target)
    print(f"'{aml_compute_target}' Compute Target already exist.")
    
except ComputeTargetException as ComputeNotFound:
    print(ComputeNotFound.message)
    #Method-1
    # print("Creating new ComputeTarget :",aml_compute_target)
    # provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
    #                                                         min_nodes = 1, 
    #                                                         max_nodes = 4,
    #                                                         idle_seconds_before_scaledown=3000)    
    # aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    # aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    #Method-2
    # compute = AmlCompute(name=cluster_name, size='STANDARD_DS3_V2',
    #                      max_instances=4)
    # cluster = ml_client.compute.begin_create_or_update(compute)
    # print("Created")
except Exception as e:
    print(e)

'buf-batch-cluster' Compute Target already exist.


In [ ]:
# Register data
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

web_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
data_name = "creditcard_defaults"
credit_data = Data(
    name=data_name,
    path=web_path,
    type=AssetTypes.URI_FILE,
    description="Dataset for credit card defaults",
    tags={"source_type": "web", "source": "UCI ML Repo"},
    version="1.0.0",
)

In [ ]:
# Registering the data to your workspace so it becomes reusable across pipelines.
credit_data = ml_client.data.create_or_update(credit_data)
print(f"Dataset with name {credit_data.name} was registered to workspace, the dataset version is {credit_data.version}")

# In the future, you can fetch the same dataset from the workspace using credit_dataset = ml_client.data.get("<DATA ASSET NAME>", version='<VERSION>').
# Azure Machine Learning mounts datasets as folders to the computes, therefore, we created an auxiliary select_first_file function to access the data file inside the mounted input folder.
credit_dataset = ml_client.data.get(data_name, version=credit_data.version)
print(credit_dataset)

Dataset with name creditcard_defaults was registered to workspace, the dataset version is 1.0.0
creation_context:
  created_at: '2023-03-23T11:47:09.716646+00:00'
  created_by: Mohammad Affan
  created_by_type: User
  last_modified_at: '2023-03-23T11:47:09.726126+00:00'
description: Dataset for credit card defaults
id: /subscriptions/0f644ff4-434a-4446-a414-16ddb2b713ee/resourceGroups/rg-sbihackathon/providers/Microsoft.MachineLearningServices/workspaces/ml-demo-explore/data/creditcard_defaults/versions/1.0.0
name: creditcard_defaults
path: https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
properties: {}
tags:
  source: UCI ML Repo
  source_type: web
type: uri_file
version: 1.0.0



In [ ]:
# Create a job Environment for pipeline
'''
So far, you've created a development environment on the compute instance, your development machine. 
    You'll also need an environment to use for each step of the pipeline. 
    Each step can have its own environment, or you can use some common environments for multiple steps.

In this example, you'll create a conda environment for your jobs, using a conda yaml file. 
    First, create a directory to store the file in.
'''
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
#create and write the file (conda.yml) in the dependencies directory.

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0

Overwriting ./dependencies/conda.yml


In [ ]:
# Use the yaml file to create and register this custom environment in your workspace:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.0",)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)
print(f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}")


Environment with name aml-scikit-learn is registered to workspace, the environment version is 0.1.0


In [ ]:
# Build the training pipeline, before that we'll create components and then from this component we'll create the pipeline

# Create component 1: data prep (using programmatic definition)
import os

data_prep_src_dir = "./components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)

In [ ]:
# This script performs the simple task of splitting the data into train and test datasets.

In [ ]:
%%writefile {data_prep_src_dir}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)

    credit_df = pd.read_excel(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    credit_train_df, credit_test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    credit_train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    credit_test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Writing ./components/data_prep/data_prep.py


In [ ]:
# We'll use the general purpose CommandComponent that can run command line actions. 
# This command line action can directly call system commands or run a script. 
# The inputs/outputs are specified on the command line via the ${{ ... }} notation.

from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep_credit_defaults",
    display_name="Data preparation for training",
    description="reads a .xl input, split the input to train and test",
    inputs={
        "data": Input(type="uri_folder"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=data_prep_src_dir,
    command="""python data_prep.py \
            --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
            --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [ ]:
# Create component 2: training (using yaml definition)
# We'll create will consume the training and test data, train a tree based model and return the output model. You'll use Azure Machine Learning logging capabilities to record and visualize the learning progress.
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

In [ ]:
# Create the training script in the directory:
# In this training script, once the model is trained, 
# the model file is saved and registered to the workspace. 
# Now you can use the registered model in inferencing endpoints.

In [ ]:
%%writefile {train_src_dir}/train.py
import argparse
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import os
import pandas as pd
import mlflow


def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--model", type=str, help="path to model file")
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_df = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_df = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.model, "trained_model"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Writing ./components/train/train.py


In [ ]:
# For the environment of this step, we'll use one of the built-in (curated) Azure ML environments. 
# The tag azureml, tells the system to use look for the name in curated environments.

In [ ]:
%%writefile {train_src_dir}/train.yml
# <component>
name: train_credit_defaults_model
display_name: Train Credit Defaults Model
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder
  learning_rate:
    type: number     
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >-
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --learning_rate ${{inputs.learning_rate}}
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>

Writing ./components/train/train.yml


In [ ]:
# Now create and register the component:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source=os.path.join(train_src_dir, "train.yml"))

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)
# Create (register) the component in your workspace
print(f"Component {train_component.name} with Version {train_component.version} is registered")

Uploading train (0.0 MBs): 100%|##########| 3527/3527 [00:00<00:00, 19543.03it/s]




Component train_credit_defaults_model with Version 1 is registered


In [ ]:
# Create the pipeline from components
# Now that both your components are defined and registered, you can start implementing the pipeline.
# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(compute=aml_compute_target,
            description="E2E data_perp-train pipeline",)
def credit_defaults_pipeline(pipeline_job_data_input, pipeline_job_test_train_ratio,
                        pipeline_job_learning_rate, pipeline_job_registered_model_name,):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,)

    # using train_func like a python call with its own inputs
    train_job = train_component(train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=pipeline_job_learning_rate,  # note: using a pipeline input as parameter
        registered_model_name=pipeline_job_registered_model_name,)
    
    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
    }
# Using your pipeline definition to instantiate a pipeline with your dataset, 
# split rate of choice and the name you picked for your model.
registered_model_name = "credit_defaults_model"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = credit_defaults_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=credit_data.path),
    pipeline_job_test_train_ratio=0.25,
    pipeline_job_learning_rate=0.05,
    pipeline_job_registered_model_name=registered_model_name,
)

In [41]:
# Submit the job
#  submit the job to run in Azure Machine Learning. This time you'll use create_or_update on ml_client.jobs
# We'll also pass an experiment name. 
# An experiment is a container for all the iterations one does on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure Machine Learning studio.

# Once completed, the pipeline will register a model in your workspace as a result of training.

import webbrowser

# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="e2e_registered_components",
)
# open the pipeline in web browser
webbrowser.open(pipeline_job.studio_url)

Uploading data_prep (0.0 MBs): 100%|##########| 1433/1433 [00:00<00:00, 33208.31it/s]




True

In [43]:
## DEPLOYMENT
# Deploy the model as an online endpoint

____________________________________________________________________________

In [ ]:
def get_training_dataset(dataset_id):
    '''
    Loads the previously used dataset.
    It assumes that the script is run in an AzureML command job under the same workspace as the original experiment.
    '''
    from azureml.core.dataset import Dataset
    from azureml.core.run import Run
    ws = Run.get_context().experiment.workspace
    dataset = Dataset.get_by_id(workspace=ws, id=dataset_id)
    return dataset.to_pandas_dataframe()

def split_dataset(X, y, weights, split_ratio, should_stratify):
    '''
    Splits the dataset into a training and testing set.
    Splits the dataset using the given split ratio. The default ratio given is 0.25 but can be
    changed in the main function. If should_stratify is true the data will be split in a stratified
    way, meaning that each new set will have the same distribution of the target value as the
    original dataset. should_stratify is true for a classification run, false otherwise.
    '''
    from sklearn.model_selection import train_test_split

    random_state = 42
    if should_stratify:
        stratify = y
    else:
        stratify = None

    if weights is not None:
        X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
            X, y, weights, stratify=stratify, test_size=split_ratio, random_state=random_state)
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, stratify=stratify, test_size=split_ratio, random_state=random_state)
        weights_train, weights_test = None, None
    return (X_train, y_train, weights_train), (X_test, y_test, weights_test)

def prepare_data(dataframe):
    '''
    Prepares data for training.
    Cleans the data, splits out the feature and sample weight columns and prepares the data for use in training.
    This function can vary depending on the type of dataset and the experiment task type: classification,
    regression, or time-series forecasting.
    '''
    from azureml.training.tabular.preprocessing import data_cleaning
    label_column_name = 'CustomerScore'
    
    # extract the features, target and sample weight arrays
    y = dataframe[label_column_name].values
    X = dataframe.drop([label_column_name], axis=1)
    sample_weights = None
    # Split training data into train/test datasets and take only the train dataset
    split_ratio = 0.15
    (X, y, sample_weights), _ = split_dataset(X, y, sample_weights, split_ratio, should_stratify=False)
    X, y, sample_weights = data_cleaning._remove_nan_rows_in_X_y(X, y, sample_weights,
        is_timeseries=False, target_column=label_column_name)
    
    return X, y, sample_weights

In [ ]:
def generate_data_transformation_config():
    '''
    Specifies the featurization step in the final scikit-learn pipeline.
    If you have many columns that need to have the same featurization/transformation applied (for example,
    50 columns in several column groups), these columns are handled by grouping based on type. Each column
    group then has a unique mapper applied to all columns in the group.
    '''
    from sklearn.pipeline import FeatureUnion
    column_group_0 = [['Average_Daily_Balance'], ['Current_Balance_of_Customer'], ['Credit_Limit'], ['Income_of_the_Customer']]
    column_group_3 = ['Billing_State']
    column_group_4 = ['AGE', 'Phone', 'YearOfBirth_of_Customer', 'Billing_Zip']
    column_group_2 = ['CurrentBankBranchId', 'CustomerMonthsOnBooks', 'Email_Address', 'MonthOfBirth_of_Customer', 'Billing_Address_Line_1', 'Billing_City']
    column_group_1 = ['Due_Date_Payment', 'Credit_Limit_Date']
    column_group_5 = ['Phone', 'YearOfBirth_of_Customer', 'Billing_Zip']
    
    feature_union = FeatureUnion([
        ('mapper_0', get_mapper_0(column_group_0)),
        ('mapper_1', get_mapper_1(column_group_1)),
        ('mapper_2', get_mapper_2(column_group_2)),
        ('mapper_3', get_mapper_3(column_group_3)),
        ('mapper_4', get_mapper_4(column_group_4)),
        ('mapper_5', get_mapper_5(column_group_5)),])
    return feature_union
    
def generate_preprocessor_config():
    '''
    Specifies a preprocessing step to be done after featurization in the final scikit-learn pipeline.
    Normally, this preprocessing step only consists of data standardization/normalization that is
    accomplished with sklearn.preprocessing. Automated ML only specifies a preprocessing step for
    non-ensemble classification and regression models.
    '''
    from sklearn.preprocessing import StandardScaler
    preproc = StandardScaler(
        copy=True,
        with_mean=False,
        with_std=False
    )
    return preproc

def generate_algorithm_config():
    '''
    Specifies the actual algorithm and hyperparameters for training the model.
    It is the last stage of the final scikit-learn pipeline. For ensemble models, generate_preprocessor_config_N()
    (if needed) and generate_algorithm_config_N() are defined for each learner in the ensemble model,
    where N represents the placement of each learner in the ensemble model's list. For stack ensemble
    models, the meta learner generate_algorithm_config_meta() is defined.
    '''
    from sklearn.linear_model import ElasticNet
    algorithm = ElasticNet(
        alpha=0.36905263157894735,
        copy_X=True,
        fit_intercept=True,
        l1_ratio=0.01,
        max_iter=1000,
        normalize=False,
        positive=False,
        precompute=False,
        random_state=None,
        selection='cyclic',
        tol=0.0001,
        warm_start=False
    )    
    return algorithm

def build_model_pipeline():
    '''
    Defines the scikit-learn pipeline steps.
    '''
    from sklearn.pipeline import Pipeline
    pipeline = Pipeline(
        steps=[
            ('featurization', generate_data_transformation_config()),
            ('preproc', generate_preprocessor_config()),
            ('model', generate_algorithm_config()),
        ]
    )
    return pipeline

In [ ]:
def train_model(X, y, sample_weights=None, transformer=None):
    '''
    Calls the fit() method to train the model.
    The return value is the model fitted/trained on the input data.
    '''
    model_pipeline = build_model_pipeline()
    model = model_pipeline.fit(X, y)
    return model

In [ ]:
df = get_training_dataset(training_dataset_id)
X, y, sample_weights = prepare_data(df)
split_ratio = 0.25
(X_train, y_train, sample_weights_train), (X_valid, y_valid, sample_weights_valid) = split_dataset(X, y, sample_weights, split_ratio, should_stratify=False)
model = train_model(X_train, y_train, sample_weights_train)
print(model)
# metrics = calculate_metrics(model, X, y, sample_weights, X_test=X_valid, y_test=y_valid)
# metrics_log_methods = get_metrics_log_methods()
# print(metrics)